In [0]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

news_schema = StructType(
    [
        StructField("id", StringType()),
        StructField("source_name", StringType()),
        StructField("author", StringType()),
        StructField("title", StringType()),
        StructField("content", StringType()),
        StructField("url", StringType()),
        StructField("sentiment_strategy", StringType()),
        StructField("published_at", TimestampType()),
        StructField("title_cleaned", StringType()),
        StructField("content_cleaned", StringType()),
    ]
)

In [0]:

s3_path = dbutils.widgets.get("s3_path") 
# s3_path = "s3://amzn-s3-stock-prediction/news_data/ingestion_date=2025-11-17/news_macro_politics_2025-11-17.parquet"

data_frame = spark.read.schema(news_schema).parquet(s3_path, index=False) 
# data_frame.describe().show()
# data_frame.printSchema()


In [0]:
path_table="stock_prediction.default"
table_name="articles"
data_frame.createOrReplaceTempView("source_articles_temp")

In [0]:
%sql
MERGE INTO stock_prediction.default.articles AS target
USING source_articles_temp AS source
ON target.url = source.url

WHEN MATCHED THEN
  UPDATE SET 
    target.title = source.title,
    target.source_name = source.source_name

WHEN NOT MATCHED THEN
  INSERT *